In [2]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
import NeuralNetwork as nn

2022-12-08 23:57:09.410449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 23:57:09.799714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 23:57:09.799769: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-08 23:57:09.859233: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-08 23:57:12.139395: W tensorflow/stream_executor/platform/de

In [4]:
dirname = os.path.join(os.getcwd(), 'sportimages')
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)

dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  /mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/americano 1
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/basket 9348
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/beisball 8823
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/boxeo 7752
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/ciclismo 7125
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proyecto/StochasticPerceptron/sportimages/f1 7533
/mnt/c/Users/sap98/OneDrive/Escritorio/My_Stuff/programming/Uniandes/SysBio/Proye

In [5]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

deportes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1

y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  77128
0 americano
1 basket
2 beisball
3 boxeo
4 ciclismo
5 f1
6 futbol
7 golf
8 natacion
9 tenis
Total number of outputs :  10
Output classes :  [0 1 2 3 4 5 6 7 8 9]


In [28]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (61702, 21, 28, 3) (61702,)
Testing data shape :  (15426, 21, 28, 3) (15426,)
Original label: 1
After conversion to one-hot: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
(49361, 21, 28, 3) (12341, 21, 28, 3) (49361, 10) (12341, 10)


In [29]:
train_x_fixed = train_X.reshape(49361, (21*28*3))

In [30]:
funcion_de_activacion = nn.sigmoide
red_neuronal = nn.crear_modelo_de_red([1764, 500, 28, 156, 10])
red_neuronal_entrenada = nn.entrenar_red_neuronal(red_neuronal,funcion_de_activacion,nn.mean_squared_error,valor_de_prediccion=train_x_fixed, valor_real=train_label, epochs=30, tasa_de_aprendizaje=0.8)

100%|██████████| 30/30 [04:27<00:00,  8.91s/it]


In [32]:
print(train_label[0])
y_reslt = nn.predecir(train_x_fixed[0], red_neuronal_entrenada, funcion_de_activacion)
print(y_reslt)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
